In [220]:
catchment_name = 'WykeBeck'
method_name = 'Idealised'
variable_name ='Depth'

In [221]:
from matplotlib import cm
from rasterio import plot
from mpl_toolkits.axes_grid1 import make_axes_locatable
from my_functions import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def save_clipped_to_gdf(raster, out_meta, catchment_gdf, fp):
    save_array_as_raster(raster, fp, out_meta)
    
    with rasterio.open(fp) as src:
        catchment_gdf=catchment_gdf.to_crs(src.crs)
        # print(Vector.crs)
        out_image, out_transform=mask(src,catchment_gdf.geometry,crop=True)
        out_meta=src.meta.copy() # copy the metadata of the source DEM

    out_meta.update({"driver":"Gtiff", "height":out_image.shape[1], # height starts with shape[1]
        "width":out_image.shape[2], # width starts with shape[2]
        "transform":out_transform
    })

    with rasterio.open(fp,'w',**out_meta) as dst:
        dst.write(out_image)

In [222]:
watercourses = gpd.read_file('../../../FloodModelling/ExtraData/Watercourses/WatercourseLink.shp')

In [223]:
# Specify strings relating to catchment
if catchment_name == 'LinDyke':
    catchment_name_str = "Resampled.Terrain" 
    minx, miny, maxx, maxy = 437000,  426500,  445500, 434300
    shpfile_name ='CatchmentLinDyke_exported'
    cell_size_in_m2 = 1
elif catchment_name == 'WykeBeck':
    catchment_name_str = "Terrain.wykeDEM" 
    minx, miny, maxx, maxy = 430004,  429978, 438660, 440996 
    cell_size_in_m2 = 4
    shpfile_name ='WykeBeckCatchment'

In [224]:
if method_name == 'Idealised':
    most_extreme_str ='6h_sp_bl_0.9'
    least_extreme_str ="6h_sp_fl_0.1"
elif method_name == 'Observed':
    most_extreme_str ='6h_c5'
    least_extreme_str="6h_c8"

### Read catchment shapefile

In [225]:
model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles/'.format(catchment_name, method_name)
catchment_shp = model_directory + "{}.shp".format(shpfile_name)
catchment_gdf = gpd.read_file(catchment_shp)

### Read in flood data

In [226]:
most_extreme = rxr.open_rasterio(model_directory+ f"{most_extreme_str}/{variable_name} (Max).{catchment_name_str}.tif",masked=True).squeeze()
least_extreme = rxr.open_rasterio(model_directory+ f"{least_extreme_str}/{variable_name} (Max).{catchment_name_str}.tif",masked=True).squeeze()

### Get out meta for proper saving

In [227]:
out_meta = rasterio.open(model_directory+ f"{most_extreme_str}/{variable_name} (Max).{catchment_name_str}.tif").meta.copy()

### Make versions where cells with flooding <0.1m are set to np.nan

In [228]:
most_extreme = np.where(most_extreme > 0.1, most_extreme, np.nan)
least_extreme = np.where(least_extreme > 0.1, least_extreme, np.nan)

In [229]:
fp = model_directory + f"CalculatedLayers/{variable_name}_{most_extreme_str}_over_0.1m.tif"
save_clipped_to_gdf(most_extreme, out_meta, catchment_gdf, fp)

fp = model_directory + f"CalculatedLayers/{variable_name}_{least_extreme_str}_over_0.1m.tif"
save_clipped_to_gdf(least_extreme, out_meta, catchment_gdf, fp)

### Make version where cells with flooding are set to 1, and all others are set to np.nan

In [230]:
most_extreme = np.where(most_extreme > 0.1, 1, np.nan)
least_extreme = np.where(least_extreme > 0.1, 1, np.nan)

### Set any NA values to 0, where the other scenario doesn't have an NaN
This allows the difference to be counted for cells which have an NaN in one but not the other

In [231]:
most_extreme[np.isnan(most_extreme) & (~np.isnan(least_extreme))] = 0
least_extreme[np.isnan(least_extreme) & (~np.isnan(most_extreme))] = 0

### Label each cell according to whether it has flooding in neither scenario, both scenarios or 1 or the other

In [232]:
# Flooded in neither scenario
flooded_by_scenario = np.where((np.isnan(most_extreme)& np.isnan(least_extreme)), np.nan, 
        # 0 = flooded in both scenarios (over 0.1m)
        np.where(((least_extreme ==1) & (most_extreme ==1)), 0,
            # 1 = flooded in most_extreme, but not least_extreme
            np.where(((most_extreme ==1) & (least_extreme ==0)), 1,
            np.where(((most_extreme ==1) & np.isnan(least_extreme)), 1,             
                # 2 = flooded in least_extreme, but not most_extreme
                np.where(((least_extreme ==1) & (most_extreme ==0)), 2 ,
                np.where(((least_extreme ==1) & np.isnan(most_extreme)), 2 ,                 
                    # 500 = any other cells (shouldn't be any)        
                     600))))))

In [233]:
fp = model_directory + f"CalculatedLayers/{variable_name}_{method_name}_Flooded_by_scenario.tif"
save_clipped_to_gdf(flooded_by_scenario, out_meta, catchment_gdf, fp)

In [234]:
# # Find difference as either a positive or a negative
# pos_neg_depth_diff = np.where((np.isnan(diff)), np.nan, 
#                       np.where((diff < 0), 0,
#                         np.where((diff == 0), 1,
#                             np.where((diff > 0), 2, 500))))
# save_array_as_raster(pos_neg_depth_diff, 
#                      model_directory + f"CalculatedLayers/PosNegDifferencesBetweenExtremes_{variable_name}_{method_name}.tif", out_meta) 


# def plot_difference_between_extremes(catchment_dict, variable_name, method_name, ax,add_basemap= True, add_colorbar= False):
#     #######################################
#     # Read in catchment shapefile
#     #######################################
#     model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles/'.format(catchment_dict['name'], method_name)
#     catchment_shp = model_directory + "{}.shp".format(catchment_dict["shpfile_name"])
#     catchment_gdf = gpd.read_file(catchment_shp)
#     print(catchment_gdf.crs.to_string())
    
#     #######################################
#     # Define colormap
#     #######################################
#     cmap = mpl.cm.RdYlGn.copy()
#     #cmap.set_bad('purple', 1.)
        
#     #######################################
#     # Plot catchment boundary and basemap
#     #######################################
#     catchment_gdf.plot(ax=ax, facecolor="none", linewidth=5, edgecolor = 'black')
#     if add_basemap == True:
#         cx.add_basemap(ax, crs = catchment_gdf.crs.to_string(), url = cx.providers.Stamen.Watercolor,alpha=0.2)
    
#     #######################################
#     # Read in data and add to map
#     #######################################
#     fp = model_directory + f"CalculatedLayers/DifferencesBetweenExtremes_{variable_name}_{method_name}.tif"
    
#     with rasterio.open(fp) as src:
#         #array = src.read(1)
#         image = plot.show(src, ax=ax,  cmap=cmap,  vmin=-0.1,  vmax=0.1)

#     #######################################
#     # Format
#     #######################################
#     #ax.axis('off')
#     ax.set_title(method_name, fontsize =20)
    
#     # Colorbar creation
#     if add_colorbar == True:  
#         if add_basemap == True:
#             im = image.get_images()[1]
#         else:
#             im = image.get_images()[0]
#         divider = make_axes_locatable(ax)
#         cax = divider.append_axes("right", size="8%", pad=0.09)
#         cbar  = fig.colorbar(im, cax)
        
#         cbar.ax.tick_params(labelsize=50)

### Calculate layer values
How to plot in QGIS:
- Properties -> Symbology -> Paletted/unique values -> Classify

In [235]:
# cmaplist = ['powderblue', 'darkred', 'darkblue']
# cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist,3)
# cmap

In [236]:
# catchment_dict=wb_dict
# method_name='Idealised'
# variable_name='Depth'
# model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles/'.format(catchment_dict['name'], method_name)
# fp = model_directory + f"CalculatedLayers/FloodedMostExtremeOnly_{variable_name}_{method_name}.tif"

# #######################################
# model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles/'.format(catchment_dict['name'], method_name)
# catchment_shp = model_directory + "{}.shp".format(catchment_dict["shpfile_name"])
# catchment_gdf = gpd.read_file(catchment_shp)

# watercourses_clip = gpd.clip(watercourses, catchment_gdf)

# THIS IS THE ONE

In [237]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# img = mpimg.imread('../test.png')
# fig, ax = plt.subplots(figsize=(8, 8))
# imgplot = ax.imshow(img)
# ax.axis('off')

# patches_list = []
# labels = ['Both', 'BL only', 'FL only']
# for i, color in  enumerate(cmaplist):
#     patch =  mpatches.Patch(color=color, label=labels[i])
#     patches_list.append(patch)  
# ax.legend(handles=patches_list, handleheight=1, handlelength=1, fontsize =10,
#          loc='upper right', bbox_to_anchor=(0.95, 0.85));
# plt.show()

# fig.savefig('../test_python.png')

In [238]:
# fig, ax = plt.subplots(figsize=(8, 8), dpi=3000)
# add_colorbar = True
# add_basemap= False

# #######################################
# fp = model_directory + f"CalculatedLayers/FloodedMostExtremeOnly_{variable_name}_{method_name}.tif"
# model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles/'.format(catchment_dict['name'], method_name)
# catchment_shp = model_directory + "{}.shp".format(catchment_dict["shpfile_name"])
# catchment_gdf = gpd.read_file(catchment_shp)

# #######################################
# # Define colormap
# #######################################
# cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist,3)

# #######################################
# # Plot catchment boundary and basemap
# #######################################
# catchment_gdf.plot(ax=ax, facecolor="none", linewidth=1, edgecolor = 'darkgreen')

# if add_basemap == True:
#     cx.add_basemap(ax, crs = catchment_gdf.crs.to_string(), url = cx.providers.Stamen.Watercolor,alpha=0.2)

# #######################################
# # Read in data and add to map
# #######################################
# with rasterio.open(fp) as src:
#     #array = src.read(1)
#     image = plot.show(src, ax=ax,  cmap=cmap)
# watercourses_clip.plot(ax=ax, facecolor="none", linewidth=0.3, edgecolor = 'blue')
# #######################################
# # Format
# #######################################
# ax.axis('off')
# #ax.set_title(method_name, fontsize =20)

# # Colorbar creation
# if add_colorbar == True:  
#     # And set up the legend object
#     patches_list = []
#     labels = ['Both', 'BL only', 'FL only']
#     for i, color in  enumerate(cmaplist):
#         patch =  mpatches.Patch(color=color, label=labels[i])
#         patches_list.append(patch)  
#     ax.legend(handles=patches_list, handleheight=1, handlelength=1, fontsize =10,
#              loc='upper right', bbox_to_anchor=(0.85, 0.85));

    
# plt.savefig('output_3.png', bbox_inches='tight', pad_inches=0.1)    
# image = Image.open('output_3.png')
# image = image.convert('P', palette=Image.ADAPTIVE, colors=2 ** 8)
# image.save('output_3_colorcompressed.png')


In [239]:
# from PIL import Image

# # Load your image using Pillow
# image = Image.open('output_3_compressed.png')
# # Reduce color depth to a specified number of bits per channel
# new_color_depth = 8  # Number of bits per channel (e.g., 8-bit)
# # Convert the image to the new color mode
# image = image.convert('P', palette=Image.ADAPTIVE, colors=2 ** new_color_depth)
# # Save the image with reduced color depth
# output_filename = 
# image.save('output_3_compressed_colorcompressed.png')


# Actually maybe this is the one

In [240]:
# cmaplist = ['khaki', 'darkred', 'darkblue']
# cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist,3)

In [241]:
# # Open the raster dataset
# fp = model_directory + f"CalculatedLayers/FloodedMostExtremeOnly_{variable_name}_{method_name}.tif"
# dataset = rasterio.open(fp)

# # Read the raster data
# data = dataset.read(1)  # Change the band number (e.g., 1, 2, 3) based on your data

# # Set up the figure size and DPI
# fig, ax = plt.subplots(figsize=(8, 8), dpi=1000)

# # Display the raster data using rasterio.plot.show
# show(data, cmap=cmap, ax=ax)

# # catchment_gdf.plot(ax=ax, facecolor="none", linewidth=5, edgecolor = 'black')

# ax.axis('off')
# # Save the figure to a file (e.g., PNG)
# plt.savefig('output.png', bbox_inches='tight', pad_inches=0.1)

### Find difference

In [242]:
# fig, axs = plt.subplots(figsize=(8, 8), dpi=1000)
# plot_difference_between_extremes(ld_dict, 'Depth', 'Idealised', axs, True, True)
# fig.savefig("Outputs/Figs/IdealisedProfiles/LinDyke/IP_LD_Diff.jpg")

In [243]:
# fig, axs = plt.subplots(ncols=1, figsize=(80, 40))
# plot_difference_between_extremes(wb_dict, 'Depth', 'Idealised', axs, True, True)

In [244]:
# fig, axs = plt.subplots(ncols=1, figsize=(80, 40))
# plot_difference_between_extremes(wb_dict, 'Depth', 'Idealised', axs, True, True)
# # plot_difference_between_extremes(wb_dict, 'Depth', 'Observed', axs[1], False, True)

In [245]:
# import pylab
# pylab.hist(diff.data[~np.isnan(diff.data)],bins=30);

In [246]:
# def colorize(array, cmap, vmin, vmax):
#     cm = plt.cm.get_cmap(cmap)    
#     if np.nanmin(array) == np.nanmax(array):
#         return np.uint8(cm(array)  * 255)
#     else:
#         normed_data = (array - vmin) / (vmax - vmin) 
#         # normed_data = (array - np.nanmin(array)) / (np.nanmax(array) - np.nanmin(array)) 
#         return np.uint8(cm(normed_data)  * 255)

# def folium_baseplot(catchment_dict, method_name):
    
#     model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles/'.format(catchment_dict['name'], method_name)
#     catchment_shp = model_directory + "{}.shp".format(catchment_dict["shpfile_name"])
#     catchment_gdf = gpd.read_file(catchment_shp)
    
#     # Set up figure
#     f = folium.Figure(width=1000, height=1000)

#     # Create base map - location figures were from clat, clon, but wanted to create map before loop
#     mapa = folium.Map(location=catchment_dict['centre_location'],zoom_start=13,tiles ='cartodb positron').add_to(f)
#     folium.TileLayer(tiles = 'https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}',
#         attr="No Base Map",name="No Base Map",show=True).add_to(mapa)

#     # Catchment boundary
#     catchment_boundary_feature_group = FeatureGroup(name=catchment_dict['name'])
#     catchment_boundary_feature_group.add_child(folium.GeoJson(data=catchment_gdf["geometry"], style_function=lambda x, 
#                                          fillColor='#00000000', color='green': { "fillColor": '#00000000',"color": 'Black',}))
#     mapa.add_child(catchment_boundary_feature_group)
        
#     return mapa

# def add_folium_layer(mapa, fp, layername):

#     ###########################################
#     # Read in data
#     ###########################################
#     with rasterio.open(fp) as src:
#         array = src.read(1)
#     layer=prep_for_folium_plotting (fp)
    
#     ###########################################
#     # Get coordinates needed in plotting
#     ###########################################
#     clat, clon = layer.y.values.mean(), layer.x.values.mean()
#     mlat, mlon = layer.y.values.min(), layer.x.values.min()
#     xlat, xlon = layer.y.values.max(), layer.x.values.max()
    
#     ###########################################
#     #  Add to feature group
#     ###########################################
#     data  = ma.masked_invalid(layer)
#     colored_data = colorize(data.data, cmap='RdYlGn', vmin = -0.1,vmax= 0.1)
#     feature_group = FeatureGroup(name=layername)
#     feature_group.add_child(folium.raster_layers.ImageOverlay(colored_data,[[mlat, mlon], [xlat, xlon]],
#                               opacity=0.8,interactive=True, popup='Test'))
#     mapa.add_child(feature_group)
    
#     return mapa

# def plot_with_folium(catchment_dict, variable_name, method_name, fps_and_names):
    
#     # Create basemap
#     mapa = folium_baseplot(catchment_dict, method_name)
    
#     # Get fp of layer to add to map
#     for name, fp  in fps_and_names.items():
#         # Add layer to map 
#         mapa = add_folium_layer(mapa, fp, name)
   
#     # # Add layers to map
#     mapa.add_child(LayerControl("topright", collapsed = False))
#     # Display the map
#     display(mapa)
#     img_data = mapa._to_png(5)
#     img = Image.open(io.BytesIO(img_data))
#     img.save("Outputs/Figs/IdealisedProfiles/LinDyke/test.png")
    
#     return mapa

In [247]:
# method_name = 'Idealised'
# methods_dict = idealised_dict
# variable_name ='Depth'
# catchment_dict = wb_dict
# model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles/'.format(catchment_dict['name'], method_name)
    
# fps_and_names = {#'diff':model_directory + f"CalculatedLayers/DifferencesBetweenExtremes_{variable_name}_{method_name}.tif",
# 'difference':model_directory + f"CalculatedLayers/FloodedMostExtremeOnly_{variable_name}_{method_name}.tif",                 
# #         'posnegdiff':model_directory + f"CalculatedLayers/PosNegDifferencesBetweenExtremes_{variable_name}_{method_name}.tif",
# #                 'most_extreme':model_directory+ f"{methods_dict['most_extreme']}/{variable_name} (Max).{catchment_dict['name_str']}.tif",
#                 # 'least_extreme':model_directory+ f"{methods_dict['least_extreme']}/{variable_name} (Max).{catchment_dict['name_str']}.tif",
#                 }

# mapa = plot_with_folium(wb_dict, 'Depth', 'Idealised', fps_and_names)

In [248]:
# # Open fire boundary data with geopandas
# catchment_boundary = gpd.read_file(fire_boundary_path)

### This makes a plot which is too big

In [250]:
# import earthpy.spatial as es
# from rasterio.plot import plotting_extent
# import earthpy.plot as ep

# fp = model_directory + f"CalculatedLayers/FloodedMostExtremeOnly_{variable_name}_{method_name}.tif"
# fig, axs = plt.subplots(ncols=1, figsize=(40, 40))
# #################################################################
# # Read in and prepare data
# #################################################################
# # # Read in shapefile for use in cropping
# # with rasterio.open(fp) as raster_crs:
# #     # Get crs to read in the catchment shape file
# #     crop_raster_profile = raster_crs.profile
# #     crop_bound_utm13N = catchment_gdf.to_crs(crop_raster_profile["crs"])    

# # watercourses_clip.to_crs(crop_raster_profile["crs"])  

# # Read in raster data amd crop to the shapefile
# with rasterio.open(fp) as src:
#     single_cropped_image, single_cropped_meta = es.crop_image(src, catchment_gdf)
#     #single_cropped_image = np.ma.masked_array(single_cropped_image, single_cropped_image < 0)
    
# # Create the extent object for plotting
# single_crop_extent = plotting_extent(single_cropped_image[0], single_cropped_meta["transform"])

# # cx.add_basemap(axs, crs = catchment_gdf.crs.to_string(),
# #                url =cx.providers.OpenStreetMap.Mapnik,zoom="auto",alpha=0.2)    

# #################################################################
# # Mask array if necessary (in this case this is for the terrain data)
# #################################################################
# # # For landcover plot
# # if colors != None:
# #     # Mask array (don't want to include values of 0, as these are no data values here)
# #     single_cropped_image = np.ma.masked_array(single_cropped_image[0], single_cropped_image[0] < 0.001)
# #     # Define a vmin and vmax (so it uses the values for this plot)
# #     vmin = np.nanmin(single_cropped_image)
# #     vmax = np.nanmax(single_cropped_image)

# # For terrain plot
# # Define a vmin and vmax (so it's consistent across the plots)
# vmin = 0
# vmax = 3


# # And set up the legend object
# patches_list = []
# labels = ['Both', 'BL only', 'FL only']
# for i, color in  enumerate(cmaplist):
#     patch =  mpatches.Patch(color=color, label=labels[i])
#     patches_list.append(patch)  
# axs.legend(handles=patches_list, handleheight=2, handlelength=2, fontsize =20);

# #################################################################
# # Plot the image
# #################################################################
# # crop_bound_utm13N.boundary.plot(ax=axs, color="black", zorder=10)
# ep.plot_bands(single_cropped_image, ax=axs, cmap = cmap, 
#                   vmin=vmin,vmax=vmax,extent=single_crop_extent,cbar=False)


# catchment_gdf.plot(ax=axs, facecolor="none", linewidth=3, edgecolor = 'black')
# watercourses_clip.plot(ax=axs, facecolor="none", linewidth=2, edgecolor = 'blue')

# fig.savefig("Outputs/Figs/IdealisedProfiles/LinDyke/earthpy.jpg", dpi=300)    